In [ ]:
# Databricks notebook source
# =============================================================================
# 🎯 MODEL EVALUATION SCRIPT (EVALUATION ONLY - NO REGISTRATION)
# =============================================================================
# Purpose: Find best model from experiment and prepare for registration
# Fixed: Completely rewritten to avoid aliases iteration issues
# =============================================================================

%pip install xgboost requests
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime
from pyspark.sql import SparkSession
import traceback
import json

print("=" * 80)
print("🎯 MODEL EVALUATION SYSTEM (EVALUATION ONLY)")
print("=" * 80)

# =============================================================================
# ✅ CONFIGURATION (MUST MATCH TRAINING SCRIPT)
# =============================================================================
EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

MODEL_ARTIFACT_PATH = "xgboost_model"
METRIC_KEY = "test_rmse"
IMPROVEMENT_THRESHOLD = 0.02  # 2% improvement required

# Delta Tables
EVALUATION_LOG_TABLE = "workspace.default.model_evaluation_log"
BEST_MODEL_METADATA_TABLE = "workspace.default.best_model_metadata"

print("\n📋 CONFIGURATION:")
print(f"   Experiment: {EXPERIMENT_NAME}")
print(f"   Model Name: {MODEL_NAME}")
print(f"   Metric: {METRIC_KEY} (lower is better)")
print(f"   Threshold: {IMPROVEMENT_THRESHOLD * 100}%")
print("=" * 80)

# =============================================================================
# ✅ INITIALIZATION
# =============================================================================
try:
    spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    print("✅ MLflow and Spark initialized\n")

    # Verify experiment exists
    exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if exp is None:
        print(f"❌ ERROR: Experiment '{EXPERIMENT_NAME}' not found!")
        print("\n💡 Available experiments:")
        all_exps = client.search_experiments(max_results=20)
        for e in all_exps:
            print(f"   - {e.name}")
        sys.exit(1)
    
    print(f"✅ Experiment found: {EXPERIMENT_NAME}")
    print(f"   Experiment ID: {exp.experiment_id}\n")

except Exception as e:
    print(f"❌ Initialization failed: {e}")
    traceback.print_exc()
    sys.exit(1)


# =============================================================================
# 📊 STEP 1: GET BEST MODEL FROM EXPERIMENT
# =============================================================================
def get_best_model_from_experiment():
    """Find the best performing model from all experiment runs"""
    print(f"\n{'='*70}")
    print("📋 STEP 1: Finding BEST Model From Experiment")
    print(f"{'='*70}")

    try:
        exp = client.get_experiment_by_name(EXPERIMENT_NAME)
        
        # Get all runs sorted by RMSE (ascending = best first)
        all_runs = client.search_runs(
            [exp.experiment_id],
            filter_string=f"metrics.{METRIC_KEY} > 0",
            order_by=[f"metrics.{METRIC_KEY} ASC"],
            max_results=1000
        )

        if not all_runs:
            print(f"\n❌ ERROR: No runs found with valid '{METRIC_KEY}' metric!")
            print("\n💡 Please run training_script.py first")
            return None

        print(f"✅ Total runs in experiment: {len(all_runs)}")

        # Show top 10 models
        print(f"\n📊 Top 10 Models (by {METRIC_KEY}):")
        print(f"{'Rank':<6} {'Run Name':<40} {'RMSE':<15} {'Timestamp':<20}")
        print("-" * 100)

        for i, run in enumerate(all_runs[:10], 1):
            run_name = run.info.run_name or "Unnamed"
            metric_val = run.data.metrics.get(METRIC_KEY, float('inf'))
            timestamp = datetime.fromtimestamp(run.info.start_time/1000).strftime('%Y-%m-%d %H:%M')
            marker = "👑 BEST" if i == 1 else f"{i}."
            print(f"{marker:<6} {run_name:<40} {metric_val:<15.6f} {timestamp}")

        # Select best model
        best_run = all_runs[0]
        run_id = best_run.info.run_id
        run_name = best_run.info.run_name or "Unnamed"
        metrics = best_run.data.metrics
        params = best_run.data.params
        metric_value = metrics.get(METRIC_KEY)

        print(f"\n✅ BEST Model Selected:")
        print(f"   Run ID: {run_id}")
        print(f"   Run Name: {run_name}")
        print(f"   {METRIC_KEY}: {metric_value:.6f}")
        print(f"   Rank: #1 out of {len(all_runs)} runs")
        print(f"   Timestamp: {datetime.fromtimestamp(best_run.info.start_time/1000)}")

        return {
            'run_id': run_id,
            'run_name': run_name,
            'metric_key': METRIC_KEY,
            'metric_value': metric_value,
            'params': params,
            'all_metrics': metrics,
            'timestamp': best_run.info.start_time,
            'total_runs': len(all_runs),
            'model_uri': f"runs:/{run_id}/{MODEL_ARTIFACT_PATH}",
            'artifact_path': MODEL_ARTIFACT_PATH
        }

    except Exception as e:
        print(f"❌ Error getting best model: {e}")
        traceback.print_exc()
        return None


# =============================================================================
# 🔧 HELPER: GET MODEL ALIASES SAFELY
# =============================================================================
def get_model_aliases_safe(model_name, version):
    """
    Safely get aliases for a model version using direct API call
    Returns list of alias strings
    """
    try:
        # Use get_model_version_by_alias to check each common alias
        common_aliases = ['production', 'Staging', 'champion', 'baseline']
        found_aliases = []
        
        for alias in common_aliases:
            try:
                # Try to get version by this alias
                alias_version = client.get_model_version_by_alias(model_name, alias)
                if alias_version and str(alias_version.version) == str(version):
                    found_aliases.append(alias)
            except:
                # Alias doesn't exist or doesn't point to this version
                continue
        
        return found_aliases
    except Exception as e:
        # If all fails, return empty list
        return []


# =============================================================================
# 🏆 STEP 2: GET CURRENT REGISTERED MODEL (COMPLETELY REWRITTEN)
# =============================================================================
def get_current_registered_model():
    """Get current registered model from registry - completely rewritten to avoid aliases issues"""
    print(f"\n{'='*70}")
    print("📋 STEP 2: Checking Current Registered Model")
    print(f"{'='*70}")

    try:
        # Search for model versions
        versions = client.search_model_versions(f"name = '{MODEL_NAME}'")
        
        if not versions:
            print("ℹ️ No models in registry (first model registration)")
            return None
        
        # Convert to list safely
        versions_list = []
        try:
            for v in versions:
                versions_list.append(v)
        except Exception as e:
            print(f"ℹ️ Unable to iterate versions: {e}")
            return None
        
        if not versions_list:
            print("ℹ️ No models in registry (first model registration)")
            return None

        print(f"✅ Found {len(versions_list)} existing version(s)")

        # Strategy: Check aliases using direct API calls
        best_version = None
        best_priority = 999  # Lower is better
        
        for v in versions_list:
            try:
                # Get aliases safely
                version_aliases = get_model_aliases_safe(MODEL_NAME, v.version)
                
                # Assign priority
                priority = 999
                if 'production' in version_aliases:
                    priority = 1
                elif 'Staging' in version_aliases:
                    priority = 2
                elif 'champion' in version_aliases:
                    priority = 3
                else:
                    priority = 10  # No alias
                
                # Keep best priority version
                if priority < best_priority:
                    best_priority = priority
                    best_version = v
                    
                    if priority == 1:
                        print(f"✅ Found Production model: Version {v.version}")
                        break  # Production is best, stop searching
                    elif priority == 2:
                        print(f"✅ Found Staging model: Version {v.version}")
            except Exception as e:
                print(f"⚠️ Error processing version {v.version}: {e}")
                continue
        
        # If no aliased version, use latest
        if best_priority == 999 and versions_list:
            best_version = versions_list[0]
            print(f"✅ Using latest model: Version {best_version.version}")

        if best_version:
            try:
                # Get run details
                run = client.get_run(best_version.run_id)
                metric = run.data.metrics.get(METRIC_KEY)
                
                # Get aliases one more time for display
                final_aliases = get_model_aliases_safe(MODEL_NAME, best_version.version)
                
                print(f"   Version: {best_version.version}")
                print(f"   Run ID: {best_version.run_id}")
                print(f"   {METRIC_KEY}: {metric:.6f}" if metric else "   Metric: N/A")
                print(f"   Aliases: {', '.join(final_aliases) if final_aliases else 'None'}")
                
                return {
                    'version': best_version.version,
                    'run_id': best_version.run_id,
                    'metric_value': metric if metric else 0.0,
                    'aliases': final_aliases
                }
                
            except Exception as e:
                print(f"⚠️ Error fetching run details: {e}")
                return None
        
        return None

    except Exception as e:
        print(f"ℹ️ No registered model found: {e}")
        print("   (This is expected for first-time registration)")
        return None


# =============================================================================
# 🔍 STEP 3: EVALUATE MODEL QUALITY
# =============================================================================
def evaluate_model(new_model, current_model):
    """Evaluate if new model should be registered"""
    print(f"\n{'='*70}")
    print("📋 STEP 3: Model Evaluation")
    print(f"{'='*70}")

    # First model - automatic approval
    if not current_model:
        print("✅ APPROVED: First model (no baseline to compare)")
        return {
            'should_register': True,
            'reason': 'First model registration',
            'improvement_pct': 0.0,
            'decision': 'APPROVE'
        }

    # Compare with existing model
    new_metric = new_model['metric_value']
    current_metric = current_model['metric_value']
    
    # Handle case where current_metric might be 0 or None
    if current_metric is None or current_metric == 0:
        print("⚠️ Current model has no valid metric, approving new model")
        return {
            'should_register': True,
            'reason': 'Current model has invalid metric',
            'improvement_pct': 0.0,
            'decision': 'APPROVE'
        }
    
    improvement = (current_metric - new_metric) / current_metric
    improvement_pct = improvement * 100

    print(f"\n📊 Comparison:")
    print(f"   New Model RMSE: {new_metric:.6f}")
    print(f"   Current Model RMSE: {current_metric:.6f}")
    print(f"   Improvement: {improvement_pct:.2f}%")
    print(f"   Threshold: {IMPROVEMENT_THRESHOLD * 100}%")

    if improvement >= IMPROVEMENT_THRESHOLD:
        print(f"\n✅ APPROVED: Model improved by {improvement_pct:.2f}%")
        return {
            'should_register': True,
            'reason': f'Improvement: {improvement_pct:.2f}%',
            'improvement_pct': improvement_pct,
            'decision': 'APPROVE'
        }
    else:
        print(f"\n❌ REJECTED: Insufficient improvement ({improvement_pct:.2f}%)")
        return {
            'should_register': False,
            'reason': f'Insufficient improvement: {improvement_pct:.2f}%',
            'improvement_pct': improvement_pct,
            'decision': 'REJECT'
        }


# =============================================================================
# 💾 STEP 4: SAVE BEST MODEL METADATA
# =============================================================================
def save_best_model_metadata(model_info, evaluation_result):
    """Save best model metadata to Delta table for registration script"""
    print(f"\n{'='*70}")
    print("📋 STEP 4: Saving Best Model Metadata")
    print(f"{'='*70}")

    try:
        metadata = {
            "evaluation_timestamp": [datetime.now()],
            "run_id": [model_info['run_id']],
            "run_name": [model_info['run_name']],
            "model_uri": [model_info['model_uri']],
            "artifact_path": [model_info['artifact_path']],
            "metric_key": [model_info['metric_key']],
            "metric_value": [float(model_info['metric_value'])],
            "should_register": [bool(evaluation_result['should_register'])],
            "evaluation_reason": [str(evaluation_result['reason'])],
            "improvement_pct": [float(evaluation_result['improvement_pct'])],
            "model_name": [MODEL_NAME],
            "total_runs_evaluated": [int(model_info['total_runs'])],
            "params_json": [json.dumps(dict(model_info['params']))]
        }
        
        df = spark.createDataFrame(pd.DataFrame(metadata))
        
        # Overwrite latest evaluation result
        df.write.format("delta")\
            .mode("overwrite")\
            .option("overwriteSchema", "true")\
            .saveAsTable(BEST_MODEL_METADATA_TABLE)
        
        print(f"✅ Metadata saved to: {BEST_MODEL_METADATA_TABLE}")
        print(f"   Run ID: {model_info['run_id']}")
        print(f"   Decision: {evaluation_result['decision']}")
        
        return True

    except Exception as e:
        print(f"❌ Failed to save metadata: {e}")
        traceback.print_exc()
        return False


# =============================================================================
# 📝 STEP 5: LOG EVALUATION HISTORY
# =============================================================================
def log_evaluation_history(model_info, current_model, evaluation_result):
    """Log evaluation to history table"""
    print(f"\n{'='*70}")
    print("📋 STEP 5: Logging Evaluation History")
    print(f"{'='*70}")

    try:
        log_data = {
            "timestamp": [datetime.now()],
            "new_run_id": [model_info['run_id']],
            "new_run_name": [model_info['run_name']],
            "new_metric": [float(model_info['metric_value'])],
            "current_version": [int(current_model['version']) if current_model else None],
            "current_metric": [float(current_model['metric_value']) if current_model and current_model['metric_value'] else None],
            "current_alias": ['Staging' if current_model else None],
            "should_promote": [bool(evaluation_result['should_register'])],
            "promotion_reason": [str(evaluation_result['reason'])],
            "improvement_pct": [float(evaluation_result['improvement_pct'])],
            "promoted_to_staging": [False],  # Will be updated by registration script
            "promoted_version": [None],  # Will be updated by registration script
            "threshold_used": [float(IMPROVEMENT_THRESHOLD * 100)],
            "total_runs_evaluated": [int(model_info['total_runs'])],
            "selection_method": ["ALL-TIME BEST"]
        }
        
        df = spark.createDataFrame(pd.DataFrame(log_data))
        
        df.write.format("delta")\
            .mode("append")\
            .option("mergeSchema", "true")\
            .saveAsTable(EVALUATION_LOG_TABLE)
        
        print(f"✅ History logged to: {EVALUATION_LOG_TABLE}")

    except Exception as e:
        print(f"⚠️ Failed to log history: {e}")
        print("   (Non-critical error - continuing)")


# =============================================================================
# 🎬 MAIN EXECUTION
# =============================================================================
def main():
    print(f"\n{'='*80}")
    print("🚀 STARTING MODEL EVALUATION")
    print(f"{'='*80}")
    
    # Step 1: Find best model from experiment
    best_model = get_best_model_from_experiment()
    if not best_model:
        print("\n❌ EVALUATION FAILED - No valid models found")
        return False  # Instead of sys.exit(1)

    # Step 2: Get current registered model
    current_model = get_current_registered_model()
    
    # Step 3: Evaluate model
    evaluation_result = evaluate_model(best_model, current_model)

    # Step 4: Save metadata for registration script
    metadata_saved = save_best_model_metadata(best_model, evaluation_result)
    
    # Step 5: Log to history
    log_evaluation_history(best_model, current_model, evaluation_result)

    # Final Summary
    print("\n" + "=" * 80)
    print("✅ MODEL EVALUATION COMPLETE")
    print("=" * 80)
    print(f"📊 Selected Model:")
    print(f"   Run ID: {best_model['run_id']}")
    print(f"   Run Name: {best_model['run_name']}")
    print(f"   RMSE: {best_model['metric_value']:.6f}")
    print(f"   Rank: #1 from {best_model['total_runs']} runs")
    print(f"\n🎯 Evaluation Decision: {evaluation_result['decision']}")
    print(f"   Reason: {evaluation_result['reason']}")
    print(f"   Should Register: {'YES ✅' if evaluation_result['should_register'] else 'NO ❌'}")
    
    if metadata_saved:
        print(f"\n📦 Next Step:")
        print(f"   Run registration script to register approved model")
        print(f"   Metadata available in: {BEST_MODEL_METADATA_TABLE}")
    
    print("=" * 80)
    
    # Return True if approved, False if rejected
    return evaluation_result['should_register']


# Execute
if __name__ == "__main__":
    approved = main()
    print(f"\n🎯 MODEL APPROVAL STATUS: {'APPROVED ✅' if approved else 'REJECTED ❌'}")

    
    
    

# # Databricks notebook source
# # =============================================================================
# # 🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM
# # =============================================================================
# # This script compares newly trained model with current best model
# # Auto-promotes if better, sends notifications, logs everything
# # =============================================================================

# %pip install xgboost requests
# import mlflow
# from mlflow.tracking import MlflowClient
# import pandas as pd
# import numpy as np
# import json
# import sys
# import os
# from datetime import datetime
# from pyspark.sql import SparkSession
# import requests
# import traceback

# print("=" * 80)
# print("🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM")
# print("=" * 80)

# # =============================================================================
# # ✅ CONFIGURATION (ALIGNED WITH TRAINING SCRIPT)
# # =============================================================================
# EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"
# MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

# STAGING_ALIAS = "staging"   # 🔄 aligned lowercase alias for consistency
# PRODUCTION_ALIAS = "production"

# MODEL_ARTIFACT_PATH = "xgboost_model"   # ✅ exactly same as training script

# METRIC_KEY = "test_rmse"
# IMPROVEMENT_THRESHOLD = 0.02  # 2% improvement needed for promotion

# # Logging Config
# COMPARISON_LOG_TABLE = "workspace.default.model_evaluation_log"

# # =============================================================================
# # ✅ INITIALIZATION
# # =============================================================================
# try:
#     spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
#     mlflow.set_tracking_uri("databricks")
#     mlflow.set_registry_uri("databricks-uc")
#     client = MlflowClient()
#     print("✅ MLflow and Spark initialized\n")
# except Exception as e:
#     print(f"❌ Initialization failed: {e}")
#     sys.exit(1)


# # =============================================================================
# # 📊 STEP 1: GET LATEST TRAINED MODEL FROM EXPERIMENT
# # =============================================================================
# def get_latest_trained_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 1: Finding Latest Trained Model (Metric-driven)")
#     print(f"{'='*70}")

#     try:
#         exp = client.get_experiment_by_name(EXPERIMENT_NAME)
#         if not exp:
#             raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found")

#         runs = client.search_runs(
#             [exp.experiment_id],
#             order_by=["metrics." + METRIC_KEY + " DESC"],  # Fetch best metric, not latest timestamp
#             max_results=1
#         )

#         if not runs:
#             raise ValueError("No runs found in experiment")

#         best_run = runs[0]
#         run_id = best_run.info.run_id
#         run_name = best_run.info.run_name or "Unnamed"
#         metrics = best_run.data.metrics
#         params = best_run.data.params
#         metric_value = metrics.get(METRIC_KEY)

#         print(f"\n✅ Best Training Run Found (by {METRIC_KEY}):")
#         print(f"   Run ID: {run_id}")
#         print(f"   Run Name: {run_name}")
#         print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#         print(f"   Parameters: {dict(list(params.items())[:3])}...")
#         print(f"   Timestamp: {datetime.fromtimestamp(best_run.info.start_time/1000)}")

#         return {
#             'run_id': run_id,
#             'run_name': run_name,
#             'metric': metric_value,
#             'params': params,
#             'metrics_all': metrics,
#             'timestamp': best_run.info.start_time
#         }

#     except Exception as e:
#         print(f"❌ Error getting best model: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 🏆 STEP 2: GET CURRENT BEST MODEL (STAGING/PRODUCTION)
# # =============================================================================
# def get_current_best_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 2: Finding Current Best Model in Registry")
#     print(f"{'='*70}")

#     best_model = None
#     for alias_name in [PRODUCTION_ALIAS, STAGING_ALIAS]:
#         try:
#             mv = client.get_model_version_by_alias(MODEL_NAME, alias_name)
#             run = client.get_run(mv.run_id)
#             metric_value = run.data.metrics.get(METRIC_KEY)
#             if metric_value is None:
#                 metric_tag = mv.tags.get("metric_rmse")
#                 metric_value = float(metric_tag) if metric_tag else None

#             best_model = {
#                 'version': mv.version,
#                 'run_id': mv.run_id,
#                 'alias': alias_name,
#                 'metric': metric_value,
#                 'params': run.data.params,
#                 'metrics_all': run.data.metrics
#             }

#             print(f"\n✅ Found Model with @{alias_name} Alias:")
#             print(f"   Version: v{mv.version}")
#             print(f"   Run ID: {mv.run_id}")
#             print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#             break
#         except Exception:
#             print(f"   No model found with @{alias_name} alias")
#             continue

#     if not best_model:
#         print("\nℹ️ No existing model in registry. This will be the first model.")

#     return best_model


# # =============================================================================
# # ⚖️ STEP 3: COMPARE MODELS
# # =============================================================================
# def compare_models(new_model, current_model):
#     print(f"\n{'='*70}")
#     print("📋 STEP 3: Model Comparison Analysis")
#     print(f"{'='*70}")

#     if current_model is None:
#         print("\n🟢 DECISION: PROMOTE — First model, no existing baseline.")
#         return True, "First model - no comparison needed", None

#     if new_model['metric'] is None:
#         print("\n🔴 DECISION: DO NOT PROMOTE — Missing new model metric.")
#         return False, "New model missing metric", None

#     if current_model['metric'] is None:
#         print("\n🟢 DECISION: PROMOTE — Current model lacks metric.")
#         return True, "Current model lacks metric", None

#     new_metric = new_model['metric']
#     current_metric = current_model['metric']

#     improvement = current_metric - new_metric
#     improvement_pct = (improvement / current_metric) * 100

#     print(f"\n📊 Comparison Summary:")
#     print(f"   New RMSE: {new_metric:.6f}")
#     print(f"   Old RMSE: {current_metric:.6f}")
#     print(f"   Improvement: {improvement:.6f} ({improvement_pct:+.2f}%)")

#     threshold_value = current_metric * IMPROVEMENT_THRESHOLD

#     if improvement > threshold_value:
#         print(f"\n🟢 PROMOTE — New model {improvement_pct:.2f}% better.")
#         return True, f"Improved by {improvement_pct:.2f}%", improvement_pct
#     elif abs(improvement) <= threshold_value:
#         print(f"\n🟡 NO PROMOTION — Similar performance.")
#         return False, f"Similar performance ({improvement_pct:+.2f}%)", improvement_pct
#     else:
#         print(f"\n🔴 DO NOT PROMOTE — Worse performance.")
#         return False, f"Worse by {abs(improvement_pct):.2f}%", improvement_pct


# # =============================================================================
# # 🚀 STEP 4: PROMOTE TO STAGING
# # =============================================================================
# def promote_to_staging(new_model, comparison_result):
#     print(f"\n{'='*70}")
#     print("📋 STEP 4: Register & Promote to Staging")
#     print(f"{'='*70}")

#     try:
#         model_uri = f"runs:/{new_model['run_id']}/{MODEL_ARTIFACT_PATH}"
#         print(f"Registering model from URI → {model_uri}")

#         new_version = mlflow.register_model(model_uri, MODEL_NAME)

#         client.set_model_version_tag(MODEL_NAME, new_version.version, "source_run_id", new_model['run_id'])
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "metric_rmse", str(new_model['metric']))
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "promotion_reason", comparison_result['reason'])

#         client.set_registered_model_alias(MODEL_NAME, STAGING_ALIAS, new_version.version)

#         print(f"\n✅ Model Registered & Promoted → @{STAGING_ALIAS}")
#         print(f"   Version: v{new_version.version}")
#         print(f"   RMSE: {new_model['metric']:.6f}")
#         print(f"   Reason: {comparison_result['reason']}")
#         return new_version.version

#     except Exception as e:
#         print(f"\n❌ Promotion failed: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 📝 STEP 5: LOG RESULTS
# # =============================================================================
# def log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version=None):
#     try:
#         log_data = {
#             'timestamp': datetime.now(),
#             'new_run_id': new_model['run_id'],
#             'new_run_name': new_model['run_name'],
#             'new_metric': new_model['metric'],
#             'current_version': int(current_model['version']) if current_model else None,
#             'current_metric': current_model['metric'] if current_model else None,
#             'current_alias': current_model['alias'] if current_model else None,
#             'should_promote': comparison_result['should_promote'],
#             'promotion_reason': comparison_result['reason'],
#             'improvement_pct': comparison_result['improvement'],
#             'promoted_to_staging': promoted_version is not None,
#             'promoted_version': int(promoted_version) if promoted_version else None,
#             'threshold_used': IMPROVEMENT_THRESHOLD * 100
#         }

#         spark.createDataFrame(pd.DataFrame([log_data])) \
#             .write.format("delta").mode("append").option("mergeSchema", "true") \
#             .saveAsTable(COMPARISON_LOG_TABLE)

#         print(f"✅ Logged to {COMPARISON_LOG_TABLE}")

#     except Exception as e:
#         print(f"⚠️ Logging failed: {e}")


# # =============================================================================
# # 🎬 MAIN EXECUTION
# # =============================================================================
# def main():
#     new_model = get_latest_trained_model()
#     if not new_model:
#         print("❌ No new model found.")
#         sys.exit(1)

#     current_model = get_current_best_model()
#     should_promote, reason, improvement = compare_models(new_model, current_model)
#     comparison_result = {
#         'should_promote': should_promote,
#         'reason': reason,
#         'improvement': improvement
#     }

#     promoted_version = promote_to_staging(new_model, comparison_result) if should_promote else None
#     log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version)

#     print("\n" + "=" * 80)
#     print("✅ MODEL EVALUATION COMPLETE")
#     print("=" * 80)
#     print(f"Decision: {'PROMOTED' if should_promote else 'NOT PROMOTED'}")
#     print(f"Reason: {reason}")
#     if promoted_version:
#         print(f"Promoted Version: v{promoted_version} → @{STAGING_ALIAS}")
#     print("=" * 80)


# # =============================================================================
# # ✅ EXECUTE
# # =============================================================================
# if __name__ == "__main__":
#     main()





# # Databricks notebook source
# # =============================================================================
# # 🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM
# # =============================================================================
# # This script compares newly trained model with current best model
# # Auto-promotes if better, sends notifications, logs everything
# # =============================================================================

# %pip install xgboost requests
# import mlflow
# from mlflow.tracking import MlflowClient
# import pandas as pd
# import numpy as np
# import json
# import sys
# import os
# from datetime import datetime
# from pyspark.sql import SparkSession
# import requests
# import traceback

# print("=" * 80)
# print("🎯 INTELLIGENT MODEL EVALUATION & AUTO-PROMOTION SYSTEM")
# print("=" * 80)

# # =============================================================================
# # ✅ CONFIGURATION (ALIGNED WITH TRAINING SCRIPT)
# # =============================================================================
# EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Config_Runs"
# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"
# MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

# STAGING_ALIAS = "staging"   # 🔄 aligned lowercase alias for consistency
# PRODUCTION_ALIAS = "production"

# MODEL_ARTIFACT_PATH = "xgboost_model"   # ✅ exactly same as training script

# METRIC_KEY = "test_rmse"
# IMPROVEMENT_THRESHOLD = 0.02  # 2% improvement needed for promotion

# # Notification & Logging Config
# ENABLE_SLACK = False
# SLACK_WEBHOOK_URL = ""
# ENABLE_EMAIL = False
# EMAIL_RECIPIENT = ""
# COMPARISON_LOG_TABLE = "workspace.default.model_evaluation_log"

# # =============================================================================
# # ✅ INITIALIZATION
# # =============================================================================
# try:
#     spark = SparkSession.builder.appName("ModelEvaluation").getOrCreate()
#     mlflow.set_tracking_uri("databricks")
#     mlflow.set_registry_uri("databricks-uc")
#     client = MlflowClient()
#     print("✅ MLflow and Spark initialized\n")
# except Exception as e:
#     print(f"❌ Initialization failed: {e}")
#     sys.exit(1)


# # =============================================================================
# # 📊 STEP 1: GET LATEST TRAINED MODEL FROM EXPERIMENT
# # =============================================================================
# def get_latest_trained_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 1: Finding Latest Trained Model")
#     print(f"{'='*70}")

#     try:
#         exp = client.get_experiment_by_name(EXPERIMENT_NAME)
#         if not exp:
#             raise ValueError(f"Experiment '{EXPERIMENT_NAME}' not found")

#         runs = client.search_runs(
#             [exp.experiment_id],
#             order_by=["start_time DESC"],
#             max_results=1
#         )

#         if not runs:
#             raise ValueError("No runs found in experiment")

#         latest_run = runs[0]
#         run_id = latest_run.info.run_id
#         run_name = latest_run.info.run_name or "Unnamed"
#         metrics = latest_run.data.metrics
#         params = latest_run.data.params
#         metric_value = metrics.get(METRIC_KEY)

#         print(f"\n✅ Latest Training Run Found:")
#         print(f"   Run ID: {run_id}")
#         print(f"   Run Name: {run_name}")
#         print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#         print(f"   Parameters: {dict(list(params.items())[:3])}...")
#         print(f"   Timestamp: {datetime.fromtimestamp(latest_run.info.start_time/1000)}")

#         return {
#             'run_id': run_id,
#             'run_name': run_name,
#             'metric': metric_value,
#             'params': params,
#             'metrics_all': metrics,
#             'timestamp': latest_run.info.start_time
#         }

#     except Exception as e:
#         print(f"❌ Error getting latest model: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 🏆 STEP 2: GET CURRENT BEST MODEL (STAGING/PRODUCTION)
# # =============================================================================
# def get_current_best_model():
#     print(f"\n{'='*70}")
#     print("📋 STEP 2: Finding Current Best Model in Registry")
#     print(f"{'='*70}")

#     best_model = None
#     for alias_name in [PRODUCTION_ALIAS, STAGING_ALIAS]:
#         try:
#             mv = client.get_model_version_by_alias(MODEL_NAME, alias_name)
#             run = client.get_run(mv.run_id)
#             metric_value = run.data.metrics.get(METRIC_KEY)
#             if metric_value is None:
#                 metric_tag = mv.tags.get("metric_rmse")
#                 metric_value = float(metric_tag) if metric_tag else None

#             best_model = {
#                 'version': mv.version,
#                 'run_id': mv.run_id,
#                 'alias': alias_name,
#                 'metric': metric_value,
#                 'params': run.data.params,
#                 'metrics_all': run.data.metrics
#             }

#             print(f"\n✅ Found Model with @{alias_name} Alias:")
#             print(f"   Version: v{mv.version}")
#             print(f"   Run ID: {mv.run_id}")
#             print(f"   {METRIC_KEY}: {metric_value:.6f}" if metric_value else f"   {METRIC_KEY}: N/A")
#             break
#         except Exception:
#             print(f"   No model found with @{alias_name} alias")
#             continue

#     if not best_model:
#         print("\nℹ️ No existing model in registry. This will be the first model.")

#     return best_model


# # =============================================================================
# # ⚖️ STEP 3: COMPARE MODELS
# # =============================================================================
# def compare_models(new_model, current_model):
#     print(f"\n{'='*70}")
#     print("📋 STEP 3: Model Comparison Analysis")
#     print(f"{'='*70}")

#     if current_model is None:
#         print("\n🟢 DECISION: PROMOTE — First model, no existing baseline.")
#         return True, "First model - no comparison needed", None

#     if new_model['metric'] is None:
#         print("\n🔴 DECISION: DO NOT PROMOTE — Missing new model metric.")
#         return False, "New model missing metric", None

#     if current_model['metric'] is None:
#         print("\n🟢 DECISION: PROMOTE — Current model lacks metric.")
#         return True, "Current model lacks metric", None

#     new_metric = new_model['metric']
#     current_metric = current_model['metric']

#     improvement = current_metric - new_metric
#     improvement_pct = (improvement / current_metric) * 100

#     print(f"\n📊 Comparison Summary:")
#     print(f"   New RMSE: {new_metric:.6f}")
#     print(f"   Old RMSE: {current_metric:.6f}")
#     print(f"   Improvement: {improvement:.6f} ({improvement_pct:+.2f}%)")

#     threshold_value = current_metric * IMPROVEMENT_THRESHOLD

#     if improvement > threshold_value:
#         print(f"\n🟢 PROMOTE — New model {improvement_pct:.2f}% better.")
#         return True, f"Improved by {improvement_pct:.2f}%", improvement_pct
#     elif abs(improvement) <= threshold_value:
#         print(f"\n🟡 NO PROMOTION — Similar performance.")
#         return False, f"Similar performance ({improvement_pct:+.2f}%)", improvement_pct
#     else:
#         print(f"\n🔴 DO NOT PROMOTE — Worse performance.")
#         return False, f"Worse by {abs(improvement_pct):.2f}%", improvement_pct


# # =============================================================================
# # 🚀 STEP 4: PROMOTE TO STAGING
# # =============================================================================
# def promote_to_staging(new_model, comparison_result):
#     print(f"\n{'='*70}")
#     print("📋 STEP 4: Register & Promote to Staging")
#     print(f"{'='*70}")

#     try:
#         model_uri = f"runs:/{new_model['run_id']}/{MODEL_ARTIFACT_PATH}"
#         print(f"Registering model from URI → {model_uri}")

#         new_version = mlflow.register_model(model_uri, MODEL_NAME)

#         client.set_model_version_tag(MODEL_NAME, new_version.version, "source_run_id", new_model['run_id'])
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "metric_rmse", str(new_model['metric']))
#         client.set_model_version_tag(MODEL_NAME, new_version.version, "promotion_reason", comparison_result['reason'])

#         client.set_registered_model_alias(MODEL_NAME, STAGING_ALIAS, new_version.version)

#         print(f"\n✅ Model Registered & Promoted → @{STAGING_ALIAS}")
#         print(f"   Version: v{new_version.version}")
#         print(f"   RMSE: {new_model['metric']:.6f}")
#         print(f"   Reason: {comparison_result['reason']}")
#         return new_version.version

#     except Exception as e:
#         print(f"\n❌ Promotion failed: {e}")
#         traceback.print_exc()
#         return None


# # =============================================================================
# # 📝 STEP 5: LOG RESULTS
# # =============================================================================
# def log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version=None):
#     try:
#         log_data = {
#             'timestamp': datetime.now(),
#             'new_run_id': new_model['run_id'],
#             'new_run_name': new_model['run_name'],
#             'new_metric': new_model['metric'],
#             'current_version': int(current_model['version']) if current_model else None,
#             'current_metric': current_model['metric'] if current_model else None,
#             'current_alias': current_model['alias'] if current_model else None,
#             'should_promote': comparison_result['should_promote'],
#             'promotion_reason': comparison_result['reason'],
#             'improvement_pct': comparison_result['improvement'],
#             'promoted_to_staging': promoted_version is not None,
#             'promoted_version': int(promoted_version) if promoted_version else None,
#             'threshold_used': IMPROVEMENT_THRESHOLD * 100
#         }

#         spark.createDataFrame(pd.DataFrame([log_data])) \
#             .write.format("delta").mode("append").option("mergeSchema", "true") \
#             .saveAsTable(COMPARISON_LOG_TABLE)

#         print(f"✅ Logged to {COMPARISON_LOG_TABLE}")

#     except Exception as e:
#         print(f"⚠️ Logging failed: {e}")


# # =============================================================================
# # 🎬 MAIN EXECUTION
# # =============================================================================
# def main():
#     new_model = get_latest_trained_model()
#     if not new_model:
#         print("❌ No new model found.")
#         sys.exit(1)

#     current_model = get_current_best_model()
#     should_promote, reason, improvement = compare_models(new_model, current_model)
#     comparison_result = {
#         'should_promote': should_promote,
#         'reason': reason,
#         'improvement': improvement
#     }

#     promoted_version = promote_to_staging(new_model, comparison_result) if should_promote else None
#     log_comparison_to_delta(new_model, current_model, comparison_result, promoted_version)

#     print("\n" + "=" * 80)
#     print("✅ MODEL EVALUATION COMPLETE")
#     print("=" * 80)
#     print(f"Decision: {'PROMOTED' if should_promote else 'NOT PROMOTED'}")
#     print(f"Reason: {reason}")
#     if promoted_version:
#         print(f"Promoted Version: v{promoted_version} → @{STAGING_ALIAS}")
#     print("=" * 80)


# # =============================================================================
# # ✅ EXECUTE
# # =============================================================================
# if __name__ == "__main__":
#     main()
